In [6]:
import os
import sys
import pandas as pd
from fuzzywuzzy import fuzz
from itertools import combinations
import csv
import glob
from datetime import datetime
import re

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================
try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    BASE_DIR = os.getcwd()

INPUT_DIR = os.path.join(BASE_DIR, "input")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

def setup_folders():
    created = False
    for folder in [INPUT_DIR, OUTPUT_DIR]:
        if not os.path.exists(folder):
            os.makedirs(folder)
            created = True
    return created

# ==========================================
# 2. DETECCIÓN Y CLASIFICACIÓN
# ==========================================

def detect_separator(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            return csv.Sniffer().sniff(f.read(4096)).delimiter
    except: return ','

def clean_filename_key(filename):
    f = filename.lower()
    f = f.replace('hubspot-properties-export-', '').replace('hubspot-crm-exports-', '').replace('all-', '')
    f = re.sub(r'\d{4}-\d{2}-\d{2}', '', f) 
    f = f.replace('.csv', '').replace('.xlsx', '').replace('.xls', '')
    return f.strip('- _')

def get_file_type(file_path):
    fname = os.path.basename(file_path).lower()
    try:
        if file_path.endswith('.csv'):
            headers = pd.read_csv(file_path, sep=detect_separator(file_path), nrows=1).columns.tolist()
        else:
            headers = pd.read_excel(file_path, nrows=1).columns.tolist()
        
        h_clean = [str(h).lower().strip() for h in headers]
        
        # --- TIPO: PROPIEDADES ---
        if 'internal name' in h_clean and 'created user' in h_clean:
            if 'contact' in fname: obj = "CONTACTS"
            elif 'compan' in fname: obj = "COMPANIES"
            elif 'deal' in fname: obj = "DEALS"
            elif 'ticket' in fname: obj = "TICKETS"
            else:
                obj = clean_filename_key(fname).upper()
            return "PROPERTIES", obj, file_path

        # --- TIPO: DATOS ---
        obj_type = None
        if 'contact' in fname: obj_type = "CONTACTS"
        elif 'compan' in fname: obj_type = "COMPANIES"
        elif 'deal' in fname: obj_type = "DEALS"
        elif 'ticket' in fname: obj_type = "TICKETS"
        else:
            if 'record id' in h_clean:
                obj_type = clean_filename_key(fname).upper()

        if obj_type:
            return "DATA", obj_type, file_path
            
    except Exception as e:
        print(f"  [X] Error leyendo {fname}: {e}")
    
    return "UNKNOWN", None, None

def scan_input_folder():
    files = glob.glob(os.path.join(INPUT_DIR, "*.*"))
    valid_files = [f for f in files if f.lower().endswith(('.csv', '.xlsx', '.xls'))]
    props, data = [], []

    print(f"\n[SCAN] Inventariando {len(valid_files)} archivos en 'input'...")
    for f in valid_files:
        ftype, obj, path = get_file_type(f)
        if ftype == "PROPERTIES":
            props.append((obj, path))
            print(f"  -> [MAPA]  {os.path.basename(f)} (ID: {obj})")
        elif ftype == "DATA":
            data.append((obj, path))
            print(f"  -> [DATOS] {os.path.basename(f)} (ID: {obj})")
    return props, data

# ==========================================
# 3. LECTURA Y ANÁLISIS
# ==========================================

def parse_percentage(value):
    try:
        if pd.isna(value): return 0.0
        return float(str(value).strip().replace('%', '')) / 100
    except: return 0.0

def load_properties_map(file_path):
    print(f"    Cargando mapa: {os.path.basename(file_path)}")
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path, sep=detect_separator(file_path), encoding='utf-8', low_memory=False)
        else:
            df = pd.read_excel(file_path)
    except: return None

    df.columns = [c.strip() for c in df.columns]
    col_map = {c.lower(): c for c in df.columns}
    
    c_int = col_map.get('internal name')
    c_user = col_map.get('created user')
    c_use = col_map.get('usages')
    c_fill = col_map.get('fill rate')

    mapping = {}
    for _, row in df.iterrows():
        internal = str(row[c_int]).strip()
        label = str(row[col_map.get('name', '')]).strip() or internal
        user = str(row[c_user]).strip()
        raw_use = str(row[c_use]).strip() if c_use and pd.notna(row[c_use]) else ""
        
        data = {
            'type': 'Default' if user == 'HubSpot' else 'Custom',
            'created_by': user,
            'is_used': (len(raw_use) > 0 and raw_use != "0" and raw_use != "nan"),
            'usage_detail': raw_use,
            'hs_fill': parse_percentage(row[c_fill]) if c_fill else 0.0
        }
        mapping[internal.lower()] = data
        mapping[label.lower()] = data
    return mapping

def analyze_file(data_path, mapping, threshold):
    fname = os.path.basename(data_path)
    print(f"    Analizando: {fname} ...")
    
    try:
        if data_path.endswith('.csv'):
            df = pd.read_csv(data_path, sep=detect_separator(data_path), encoding='utf-8', low_memory=False)
        else:
            df = pd.read_excel(data_path)
    except Exception as e:
        print(f"    [X] Error crítico: {e}")
        return None, None

    results = []
    for col in df.columns:
        key = str(col).strip().lower()
        meta = mapping.get(key)
        
        p_type = meta['type'] if meta else 'Unknown'
        user = meta['created_by'] if meta else 'Unknown'
        is_used = meta['is_used'] if meta else False
        usage_det = meta['usage_detail'] if meta else ''

        filled = df[col].count()
        unique = df[col].nunique()
        total = len(df)
        fill_pct = filled / total if total > 0 else 0
        
        # Ghost: Custom + Con Datos + Sin Uso
        is_ghost = (p_type == 'Custom') and (filled > 0) and (not is_used)
        # Monotona: Llena + Mismo Valor siempre
        is_monotone = (filled > 50) and (unique <= 1)

        results.append({
            'Property Name': col,
            'Type': p_type,
            'Created By': user,
            'HubSpot Usages': usage_det,
            'Is Ghost?': is_ghost,
            'Is Monotonic?': is_monotone,
            'Fill Rate %': round(fill_pct * 100, 2),
            'Filled Rows': filled,
            'Unique Values': unique,
            'Potential Duplicates': []
        })

    stats = pd.DataFrame(results)
    
    print(f"    Buscando duplicados (Fuzzy > {threshold}%)...")
    dups = []
    cols = stats['Property Name'].tolist()
    
    for c1, c2 in combinations(cols, 2):
        r1 = stats[stats['Property Name'] == c1].iloc[0]
        r2 = stats[stats['Property Name'] == c2].iloc[0]
        if r1['Type'] == 'Default' and r2['Type'] == 'Default': continue
        
        n1 = c1.lower().replace('_', ' ')
        n2 = c2.lower().replace('_', ' ')
        score = fuzz.ratio(n1, n2)
        
        if score >= threshold:
            dups.append({'Prop A': c1, 'Type A': r1['Type'], 'Prop B': c2, 'Type B': r2['Type'], 'Score': score})
            idx1 = stats.index[stats['Property Name'] == c1][0]
            idx2 = stats.index[stats['Property Name'] == c2][0]
            stats.at[idx1, 'Potential Duplicates'].append(f"{c2} ({score}%)")
            stats.at[idx2, 'Potential Duplicates'].append(f"{c1} ({score}%)")

    return stats, pd.DataFrame(dups)

def save_report(stats, dups, original_filename):
    clean_name = os.path.splitext(original_filename)[0]
    out_name = f"Auditoria_{clean_name}.xlsx"
    out_path = os.path.join(OUTPUT_DIR, out_name)
    
    print(f"    Generando Excel: {out_name}")
    try:
        stats['Potential Duplicates'] = stats['Potential Duplicates'].apply(lambda x: " | ".join(x))
        with pd.ExcelWriter(out_path, engine='openpyxl') as writer:
            # 1. Todas
            stats.to_excel(writer, sheet_name='Todas', index=False)
            
            # 2. NUEVA: Custom y Vacías (0 fill rate)
            # Aquí está lo que pediste: Custom + Vacías
            empty_custom = stats[(stats['Type'] == 'Custom') & (stats['Filled Rows'] == 0)]
            if not empty_custom.empty:
                empty_custom.to_excel(writer, sheet_name='Custom Empty (Delete)', index=False)
            
            # 3. Ghost (Custom + Datos + Sin Uso)
            ghosts = stats[stats['Is Ghost?'] == True].sort_values('Fill Rate %', ascending=False)
            if not ghosts.empty: 
                ghosts.to_excel(writer, sheet_name='Not Used in Automation', index=False)
            
            # 4. Duplicados
            if not dups.empty: 
                dups.sort_values('Score', ascending=False).to_excel(writer, sheet_name='Potential Duplicates', index=False)
            
            # 5. Monótonas
            monos = stats[stats['Is Monotonic?'] == True]
            if not monos.empty: 
                monos.to_excel(writer, sheet_name='Constant value', index=False)
            
        print(f"    [OK] Guardado.")
    except Exception as e:
        print(f"    [X] Error guardando: {e}")

def find_matching_property_file(data_obj_id, data_filename, props_list):
    data_key = clean_filename_key(data_filename)
    for p_obj, p_path in props_list:
        if p_obj == data_obj_id: return p_path
    for p_obj, p_path in props_list:
        if data_key in os.path.basename(p_path).lower(): return p_path
    return None

# ==========================================
# MAIN LOOP
# ==========================================

if __name__ == "__main__":
    print("==========================================")
    print("   AUDITOR V14 (CUSTOM EMPTY + MATCHING)")
    print("==========================================")
    
    if setup_folders():
        print("[!] Carpetas creadas. Pon tus archivos en 'input' y reinicia.")
        sys.exit()

    props_list, data_list = scan_input_folder()
    
    if not props_list or not data_list:
        print("\n[!] Faltan archivos en 'input'.")
        sys.exit()

    try:
        val = input("\nUmbral de similitud (Enter para 92): ").strip()
        thresh = int(val) if val else 92
    except: thresh = 92

    print("\n--- INICIANDO PROCESO ---")
    
    for obj_type, data_path in data_list:
        fname = os.path.basename(data_path)
        clean_name = os.path.splitext(fname)[0]
        expected_output = f"Auditoria_{clean_name}.xlsx"
        
        if os.path.exists(os.path.join(OUTPUT_DIR, expected_output)):
            print(f"\n[SKIP] {fname} ya fue procesado.")
            continue
        
        print(f"\n>>> Procesando: {fname} (ID: {obj_type})")
        
        best_prop_path = find_matching_property_file(obj_type, fname, props_list)
        if not best_prop_path:
            best_prop_path = props_list[0][1]
            print(f"    [!] Usando mapa genérico: {os.path.basename(best_prop_path)}")
        else:
            print(f"    (Mapa detectado: {os.path.basename(best_prop_path)})")

        mapping = load_properties_map(best_prop_path)
        if mapping:
            stats, dups = analyze_file(data_path, mapping, thresh)
            if stats is not None:
                save_report(stats, dups, fname)

    print("\n[FIN] Todo procesado.")
    input("Presiona Enter para cerrar.")

   AUDITOR V14 (CUSTOM EMPTY + MATCHING)

[SCAN] Inventariando 18 archivos en 'input'...
  -> [DATOS] all-clinic-sites.csv (ID: CLINIC-SITES)
  -> [DATOS] all-companies.csv (ID: COMPANIES)
  -> [DATOS] all-contacts.csv (ID: CONTACTS)
  -> [DATOS] all-content-resources.csv (ID: CONTENT-RESOURCES)
  -> [DATOS] all-deals.csv (ID: DEALS)
  -> [DATOS] all-leads.csv (ID: LEADS)
  -> [DATOS] all-rewards.csv (ID: REWARDS)
  -> [DATOS] all-risk-reports.csv (ID: RISK-REPORTS)
  -> [DATOS] all-tickets.csv (ID: TICKETS)
  -> [MAPA]  hubspot-properties-export-clinic-sites-2025-12-02.csv (ID: CLINIC-SITES)
  -> [MAPA]  hubspot-properties-export-companies-2025-12-01.csv (ID: COMPANIES)
  -> [MAPA]  hubspot-properties-export-contacts-2025-12-02.csv (ID: CONTACTS)
  -> [MAPA]  hubspot-properties-export-content-resources-2025-12-02.csv (ID: CONTENT-RESOURCES)
  -> [MAPA]  hubspot-properties-export-deals-2025-12-02.csv (ID: DEALS)
  -> [MAPA]  hubspot-properties-export-leads-2025-12-02.csv (ID: LEADS)
  